In [1]:
import cv2

In [2]:
import numpy as np

In [3]:
import smtplib

In [4]:
import playsound
import threading

In [5]:
Alarm_Status = False
Email_Status = False
Fire_Reported = 0

In [6]:
def play_alarm_sound_function():
    while True:
        playsound.playsound("alarm-sound.mp3", True)

In [7]:
def send_mail_function():
    recipientEmail = "Enter_Recipient_Email"
    recipientEmail = recipientEmail.lower()
    
    try:
        server = smtplib.SMTP("smtp.gmail.com", 587)
        server.ehlo()
        server.starttls()
        server.login("srishtimuskan55@gmail.com", '12@muSKan')
        server.sendmail('srishtimuskan55@gmail.com', recipientEmail, "Warning A Fire Accident has been reported on ABC Company")
        print("sent to {}".format(recipientEmail))
        server.close()
    
    except Exception as e:
        print(e)

In [8]:
video = cv2.VideoCapture("1236624770.mp4")

In [10]:
while True:
    (ret, frame) = video.read()
    
    if not ret:
        break
        
    frame = cv2.resize(frame, (1500,1200))
    
    blur = cv2.GaussianBlur(frame, (21,21), 0)
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)
    
    lower = [18, 50, 50]
    upper = [35, 255, 255]
    lower = np.array(lower, dtype = "uint8")
    upper = np.array(upper, dtype = "uint8")
    
    mask = cv2.inRange(hsv, lower, upper)
    
    output = cv2.bitwise_and(frame, hsv, mask = mask)
    
    no_red = cv2.countNonZero(mask)
    if int(no_red) > 15000:
        Fire_Reported = Fire_Reported + 1
            
    cv2.imshow("output", output)
    
    if Fire_Reported >=1:
        if Alarm_Status == False:
            threading.Thread(target= play_alarm_sound_function).start()
            Alarm_Status = True
            
        if Email_Status == False:
            threading.Thread(target = send_mail_function).start()
            Email_Status = True
            
    if cv2.waitKey(1)== ord('q'):
        break
        
cv2.destroyAllWindows()
video.release()